In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from moffragmentor import MOF

from moffragmentor.fragmentor.solventlocator import get_floating_solvent_molecules, get_all_bound_solvent_molecules
from moffragmentor.fragmentor.nodelocator import find_node_clusters, create_node_collection
from moffragmentor.fragmentor.linkerlocator import create_linker_collection

In [28]:
mof = MOF.from_cif('/Users/kevinmaikjablonka/Dropbox (LSMO)/reference_set_project/aiida_optimized_2021_01_08_3203_OLD/opt_cifs/RSM0001.cif')

In [29]:
sg = mof.structure_graph

In [30]:
sg.get_connected_sites(1)

[ConnectedSite(site=PeriodicSite: C (6.9106, 2.3102, 3.0426) [1.0600, 0.3999, 0.0235], jimage=(1, 0, 0), index=4, weight=None, dist=1.1042191098137004)]

In [31]:
unbound_solvent = get_floating_solvent_molecules(mof)
    # Find nodes
node_result = find_node_clusters(mof, unbound_solvent.indices)
node_collection = create_node_collection(mof, node_result)
    # Find bound solvent
    # 
bound_solvent = get_all_bound_solvent_molecules(mof, node_result.nodes)
    # Filter the linkers (valid linkers have at least two branch points)

2022-04-13 17:56:05.985 | WARNING  | moffragmentor.fragmentor.nodelocator:find_node_clusters:63 - No branch sites found according to branch site definition.             Using now CN=3 sites between metals as branch sites.                This is not consistent with the conventions used in other parts of the code. ̰


{6: (0, 0, 0), 0: (-1, 0, 0), 3: (0, 0, 0), 7: (0, 0, 0), 10: (0, 0, 0)} [7, 6, 3, 10] 10
{6: (0, 0, 0), 0: (0, 0, 0), 3: (0, 0, 0), 7: (0, 0, 0), 10: (0, 0, 0)} [10, 6, 3, 7] 7
{5: (0, -1, 0), 0: (0, -1, 1), 2: (1, 0, 0), 12: (0, 1, 0), 11: (0, 1, 0)} [12, 5, 2, 11] 2
{4: (0, 0, 1), 0: (-1, -1, 1), 1: (-1, 0, 0), 8: (0, 0, -1), 9: (0, 0, -1)} [9, 4, 1, 8] 1
{5: (0, -1, 0), 0: (0, -1, 0), 2: (1, 0, 0), 12: (0, 1, 0), 11: (0, 1, 0)} [11, 5, 2, 12] 2
{4: (0, 0, 1), 0: (-1, 0, 1), 1: (-1, 0, 0), 8: (0, 0, -1), 9: (0, 0, -1)} [8, 4, 1, 9] 1
{6: (0, 0, 0), 0: (-1, 0, 0), 3: (0, 0, 0), 7: (0, 0, 0), 10: (0, 0, 0)} [7, 6, 3, 10] 10
{6: (0, 0, 0), 0: (0, 0, 0), 3: (0, 0, 0), 7: (0, 0, 0), 10: (0, 0, 0)} [10, 6, 3, 7] 7
{5: (0, -1, 0), 0: (0, -1, 1), 2: (1, 0, 0), 12: (0, 1, 0), 11: (0, 1, 0)} [12, 5, 2, 11] 2
{4: (0, 0, 1), 0: (-1, -1, 1), 1: (-1, 0, 0), 8: (0, 0, -1), 9: (0, 0, -1)} [9, 4, 1, 8] 1
{5: (0, -1, 0), 0: (0, -1, 0), 2: (1, 0, 0), 12: (0, 1, 0), 11: (0, 1, 0)} [11, 5, 2, 12] 2
{4: 

In [32]:
node_result.nodes

[{0, 4, 5, 6, 7, 8, 9, 10, 11, 12}]

In [33]:
node_result.branching_indices

{4, 5, 6}

In [34]:
set(bound_solvent.indices)

set()

In [35]:
bound_solvent.molecules[0].indices

IndexError: list index out of range

In [36]:
mof.structure_graph.get_connected_sites(10)

[ConnectedSite(site=PeriodicSite: C (4.0439, 4.6877, 3.9745) [0.5000, 0.8115, 0.1885], jimage=(0, 0, 0), index=6, weight=None, dist=1.2769607847523299),
 ConnectedSite(site=PeriodicSite: Gd (7.1598, 5.1169, 4.6091) [1.0000, 0.8858, 0.1142], jimage=(0, 0, 0), index=0, weight=None, dist=2.3214169446953075)]

In [37]:
mof.structure_graph.get_connected_sites(7)

[ConnectedSite(site=PeriodicSite: C (4.0439, 4.6877, 3.9745) [0.5000, 0.8115, 0.1885], jimage=(0, 0, 0), index=6, weight=None, dist=1.2766651695954734),
 ConnectedSite(site=PeriodicSite: Gd (1.1153, 5.1169, 2.7368) [-0.0000, 0.8858, 0.1142], jimage=(-1, 0, 0), index=0, weight=None, dist=2.3212885443697715)]

In [38]:
node_result.branching_indices

{4, 5, 6}

In [39]:
linker_collection, edge_dict = create_linker_collection(
        mof, node_result, node_collection, unbound_solvent
    )

2022-04-13 17:56:07.360 | WARNING  | moffragmentor.fragmentor.linkerlocator:_create_linkers_from_node_location_result:141 - No linkers with two branching sites in molecule found. Looking for molecules with one branching site.


In [40]:
len(linker_collection)

3

In [41]:
def bridges_across_cell(mof, molecule) -> bool: 
    """Check if a molecule bridges across the cell"""
    bridges = {}

    for index in molecule.indices:
        for neighbor_site in mof.structure_graph.get_connected_sites(index):
            if not neighbor_site.index in bridges:
                bridges[neighbor_site.index] = neighbor_site.jimage
            else:
                if bridges[neighbor_site.index] == neighbor_site.jimage:
                    return True

    return False

In [42]:
linker_collection[0].show_molecule()

ImportError: AseAtomsAdaptor requires the ASE package.
Use `pip install ase` or `conda install ase -c conda-forge`

In [43]:
bridges_across_cell(mof, bbs.bound_solvent.molecules[0])

NameError: name 'bbs' is not defined

In [44]:
mof.show_structure()

ImportError: AseAtomsAdaptor requires the ASE package.
Use `pip install ase` or `conda install ase -c conda-forge`

In [45]:
linker_collection[0].smiles

'[O]C=O'

In [52]:
bbs = mof.fragment()

2022-04-13 17:57:59.741 | WARNING  | moffragmentor.fragmentor.nodelocator:find_node_clusters:63 - No branch sites found according to branch site definition.             Using now CN=3 sites between metals as branch sites.                This is not consistent with the conventions used in other parts of the code. ̰
2022-04-13 17:57:59.868 | WARNING  | moffragmentor.fragmentor.linkerlocator:_create_linkers_from_node_location_result:141 - No linkers with two branching sites in molecule found. Looking for molecules with one branching site.
2022-04-13 17:57:59.904 | INFO     | moffragmentor.fragmentor:run_fragmentation:45 - Checking for metal in linker
2022-04-13 17:57:59.904 | WARNING  | moffragmentor.fragmentor.filter:in_hull:44 - Too few points to compute Delaunay triangulation
2022-04-13 17:57:59.905 | WARNING  | moffragmentor.fragmentor.filter:in_hull:44 - Too few points to compute Delaunay triangulation
2022-04-13 17:57:59.905 | WARNING  | moffragmentor.fragmentor.filter:in_hull:44 - 

{6: (0, 0, 0), 0: (-1, 0, 0), 3: (0, 0, 0), 7: (0, 0, 0), 10: (0, 0, 0)} [7, 6, 3, 10] 10
{6: (0, 0, 0), 0: (0, 0, 0), 3: (0, 0, 0), 7: (0, 0, 0), 10: (0, 0, 0)} [10, 6, 3, 7] 7
{5: (0, -1, 0), 0: (0, -1, 1), 2: (1, 0, 0), 12: (0, 1, 0), 11: (0, 1, 0)} [12, 5, 2, 11] 2
{4: (0, 0, 1), 0: (-1, -1, 1), 1: (-1, 0, 0), 8: (0, 0, -1), 9: (0, 0, -1)} [9, 4, 1, 8] 1
{5: (0, -1, 0), 0: (0, -1, 0), 2: (1, 0, 0), 12: (0, 1, 0), 11: (0, 1, 0)} [11, 5, 2, 12] 2
{4: (0, 0, 1), 0: (-1, 0, 1), 1: (-1, 0, 0), 8: (0, 0, -1), 9: (0, 0, -1)} [8, 4, 1, 9] 1
{6: (0, 0, 0), 0: (-1, 0, 0), 3: (0, 0, 0), 7: (0, 0, 0), 10: (0, 0, 0)} [7, 6, 3, 10] 10
{6: (0, 0, 0), 0: (0, 0, 0), 3: (0, 0, 0), 7: (0, 0, 0), 10: (0, 0, 0)} [10, 6, 3, 7] 7
{5: (0, -1, 0), 0: (0, -1, 1), 2: (1, 0, 0), 12: (0, 1, 0), 11: (0, 1, 0)} [12, 5, 2, 11] 2
{4: (0, 0, 1), 0: (-1, -1, 1), 1: (-1, 0, 0), 8: (0, 0, -1), 9: (0, 0, -1)} [9, 4, 1, 8] 1
{5: (0, -1, 0), 0: (0, -1, 0), 2: (1, 0, 0), 12: (0, 1, 0), 11: (0, 1, 0)} [11, 5, 2, 12] 2
{4: 

In [56]:
bbs.linkers[0].search_pubchem()

[Compound(284)]

In [49]:
bbs.linkers[0].show_molecule()

NameError: name 'bbs' is not defined